In [163]:
import pandas as pd
import numpy as np
import datetime 

data = pd.read_csv(r'C:\Users\nadia\Dropbox\Internship - Kairaktidi\data-ypen\ΠΕΙΡΑΙΑΣ-I-PIR\PIRPYTHON.csv')
data['DATETIME']=pd.to_datetime(data['DATETIME'])

print(data)

                DATETIME   C6H6   NO2  SO2   PM10  PM2.5    O3   CO
0    2020-01-01 00:00:00   9.98  57.0  8.0  133.0  112.0   8.0  NaN
1    2020-01-01 01:00:00  11.15  56.0  8.0  142.0  120.0  10.0  NaN
2    2020-01-01 02:00:00  11.71  53.0  9.0  197.0  175.0   6.0  NaN
3    2020-01-01 03:00:00   9.26  46.0  7.0  146.0  123.0   6.0  NaN
4    2020-01-01 04:00:00   9.68  45.0  8.0  133.0  117.0   5.0  NaN
...                  ...    ...   ...  ...    ...    ...   ...  ...
8779 2020-12-31 19:00:00   0.77  32.0  4.0   44.0   12.0  40.0  1.6
8780 2020-12-31 20:00:00   1.15  52.0  5.0   46.0   16.0  26.0  2.2
8781 2020-12-31 21:00:00   0.61  36.0  5.0   47.0   21.0  43.0  2.4
8782 2020-12-31 22:00:00   0.40  52.0  4.0   25.0    6.0  28.0  1.6
8783 2020-12-31 23:00:00   0.48  41.0  6.0   27.0    8.0  38.0  0.9

[8784 rows x 8 columns]


In [176]:
# adding columns for later


data['date'] = data['DATETIME'].dt.date
data['O3rcount'] = data['O3'].rolling(8, min_periods=8).count()
data['COrcount'] = data['CO'].rolling(8, min_periods=8).count()

for i in data['O3rcount'].dropna():
    if data['O3rcount'][i] < 8:
        data['O3roll'][i] = ''

#for i in data['COrcount'].dropna():
 #   if data['COrcount'][i] < 8:
  #      data['CO'][i] = ''        
        
data['O3roll'] = data['O3'].rolling(8).mean()
data['COroll'] = data['CO'].rolling(8).mean()

print(data)

                DATETIME   C6H6   NO2  SO2   PM10  PM2.5    O3   CO  \
0    2020-01-01 00:00:00   9.98  57.0  8.0  133.0  112.0   8.0  NaN   
1    2020-01-01 01:00:00  11.15  56.0  8.0  142.0  120.0  10.0  NaN   
2    2020-01-01 02:00:00  11.71  53.0  9.0  197.0  175.0   6.0  NaN   
3    2020-01-01 03:00:00   9.26  46.0  7.0  146.0  123.0   6.0  NaN   
4    2020-01-01 04:00:00   9.68  45.0  8.0  133.0  117.0   5.0  NaN   
...                  ...    ...   ...  ...    ...    ...   ...  ...   
8779 2020-12-31 19:00:00   0.77  32.0  4.0   44.0   12.0  40.0  1.6   
8780 2020-12-31 20:00:00   1.15  52.0  5.0   46.0   16.0  26.0  2.2   
8781 2020-12-31 21:00:00   0.61  36.0  5.0   47.0   21.0  43.0  2.4   
8782 2020-12-31 22:00:00   0.40  52.0  4.0   25.0    6.0  28.0  1.6   
8783 2020-12-31 23:00:00   0.48  41.0  6.0   27.0    8.0  38.0  0.9   

            date  O3rcount  COrcount  O3roll  COroll  
0     2020-01-01       NaN       NaN     NaN     NaN  
1     2020-01-01       NaN       NaN 

In [174]:
#second dataframe for calculations

grouped_single = data.groupby('date').agg({'SO2': ['count','mean'], 'PM10': ['count','mean'], 'PM2.5': ['count','mean'], 'O3roll': ['count','max'] , 'COroll': ['count','max']})
grouped_single.replace('', np.nan).count()
grouped_single.head(10)

SO2             PM10            PM2.5            O3roll          \
           count       mean count       mean count       mean  count     max   
date                                                                           
2020-01-01    24   7.750000    24  63.500000    24  49.375000     17  29.125   
2020-01-02    24  13.125000    24  32.916667    24  15.416667     24  73.625   
2020-01-03    24  13.708333    24  28.583333    11  23.545455     24  74.875   
2020-01-04    24   5.166667    24  43.333333    24  20.500000     24  84.500   
2020-01-05    24   6.708333    24  17.500000    24   8.250000     24  81.375   
2020-01-06    24  10.208333    10  20.900000    24   9.083333     24  50.125   
2020-01-07    24  18.000000    24  46.625000    24  25.333333     24  43.125   
2020-01-08    24  13.625000    24  48.458333    24  24.375000     24  84.750   
2020-01-09    24  17.041667     0        NaN    24  20.083333     24  94.000   
2020-01-10    24  13.333333    24  18.208333     0        NaN     24  66.250   

           COroll          
            count     max  
date                       
2020-01-01      0     NaN  
2020-01-02     24  2.6500  
2020-01-03     24  1.3500  
2020-01-04     24  0.4000  
2020-01-05     24  0.4625  
2020-01-06     24  1.3875  
2020-01-07     24  1.2375  
2020-01-08     24  1.5125  
2020-01-09     24  2.2500  
2020-01-10     24  1.8125

# C6H6

In [95]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας

data.replace('', np.nan).count()
c = data['C6H6'].count()
print("Οι συνολικές τιμές είναι", data.shape[0] )
print("Οι έγκυρες τιμές είναι", c)
if c > (0.9*data.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Οι συνολικές τιμές είναι 8784
Οι έγκυρες τιμές είναι 7029
Δεν ικανοποιείται ο έλεγχος πληρότητας


In [96]:
# Έλεγχος οριακής τιμής στο ημερολογιακό έτος (5 μg/m3)

print("Μέση παρατηρούμενη ετήσια τιμή:", round(data['C6H6'].mean(), 2), "μg/m3")

if data['C6H6'].mean() > 5:
    print("Γίνεται υπέρβαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου (5 μg/m3)")
    

Μέση παρατηρούμενη ετήσια τιμή: 2.04 μg/m3
Δεν παρατηρείται υπέρβαση του ορίου (5 μg/m3)


# NO2

In [97]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας (ωριαίο)
 
print("Οι συνολικές τιμές είναι", data.shape[0] )
print("Οι έγκυρες τιμές είναι", data['NO2'].count())
if data['NO2'].count() > (0.9*data.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Οι συνολικές τιμές είναι 8784
Οι έγκυρες τιμές είναι 8765
Ικανοποιείται ο έλεγχος πληρότητας


In [98]:
# Ωριαία οριακή τιμή (200μg/m3 έως 18 φορές τον χρόνο)

if data.loc[data.NO2 > 200, 'NO2'].count() > 18 :
     print("Γίνεται υπέρβαση του ορίου")
else:
    print("Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (200μg/m3) για πάνω από 18 φορές το έτος")

Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (200μg/m3) για πάνω από 18 φορές το έτος


In [99]:
# Οριακή τιμή στο ημερολογ. έτος = 40 μg/m3

if data['NO2'].mean() > 40:
    print("Γίνεται υπέρβαση του ορίου,", round(data['NO2'].mean(), 1), "μg/m3 με όριο τα 40 μg/m3")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου ( 40 μg/m3)")

Γίνεται υπέρβαση του ορίου, 55.0 μg/m3 με όριο τα 40 μg/m3


# SO2

In [100]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας (ωριαίο)
 
print("Οι συνολικές τιμές είναι", data.shape[0] )
print("Οι έγκυρες τιμές είναι", data['SO2'].count())
if data['SO2'].count() > (0.9*data.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Οι συνολικές τιμές είναι 8784
Οι έγκυρες τιμές είναι 8487
Ικανοποιείται ο έλεγχος πληρότητας


In [101]:
# Ωριαία οριακή τιμή (350μg/m3 έως 24 φορές τον χρόνο)

if data.loc[data.SO2 > 350, 'SO2'].count() > 24 :
     print("Γίνεται υπέρβαση του ορίου")
else:
    print("Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (350μg/m3) για πάνω από 24 φορές το έτος")

print("Οι συνολικές υπερβάσεις ήταν:" ,data.loc[data.SO2 > 350, 'SO2'].count())

Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (350μg/m3) για πάνω από 24 φορές το έτος
Οι συνολικές υπερβάσεις ήταν: 0


In [102]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας για την ημερήσια οριακή τιμή (ημερήσιο)

print("Οι συνολικές τιμές είναι", grouped_single.shape[0] )
print("Οι έγκυρες τιμές είναι", len(grouped_single['SO2']) - grouped_single['SO2']['count'].le(18).sum())

if (len(grouped_single['SO2']) - grouped_single['SO2']['count'].le(18).sum()) > (0.9*data.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Οι συνολικές τιμές είναι 366
Οι έγκυρες τιμές είναι 354
Δεν ικανοποιείται ο έλεγχος πληρότητας


In [103]:
# Ημερήσια οριακή τιμή (125μg/m3 έως 3 φορές τον χρόνο)

if grouped_single['SO2']['mean'].gt(125).sum() > 3:
    print("Παρατηρείται υπέρβαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές υπερβάσεις που παρατηρήθηκαν:", grouped_single['SO2']['mean'].gt(125).sum())

Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές υπερβάσεις που παρατηρήθηκαν: 0


# PM10

In [104]:
#Έλεγχος πληρότητας (και για τα 2)
print("Συνολικές τιμές:", grouped_single.shape[0])

print("Οι έγκυρες τιμές είναι", len(grouped_single['PM10']) - grouped_single['PM10']['count'].le(18).sum())

if grouped_single['PM10']['mean'].count() > (0.9*grouped_single.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Συνολικές τιμές: 366
Οι έγκυρες τιμές είναι 321
Δεν ικανοποιείται ο έλεγχος πληρότητας


In [105]:
# Οριακή τιμή ημέρας = 50μg/m3 έως 35 φορές το έτος

if grouped_single['PM10']['mean'].gt(50).sum() > 35:
    print("Παρατηρείται υπέρβαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές υπερβάσεις που παρατηρήθηκαν:", grouped_single['PM10']['mean'].gt(50).sum())

Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές υπερβάσεις που παρατηρήθηκαν: 27


In [106]:
# Οριακή τιμή έτους = 40 μg/m3 (με βάση τις ημερήσιες τιμές και όχι τις ωριαίες)

print("Η οριακή τιμή του έτους είναι", round(grouped_single['PM10']['mean'].mean(),1), "μg/m3")

if grouped_single['PM10']['mean'].mean() > 40:
    print("Παρατηρείται παραβίαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")

Η οριακή τιμή του έτους είναι 29.8 μg/m3
Δεν παρατηρείται υπέρβαση του ορίου


# PM2.5

In [107]:
# Έλεγχος πληρότητας για το έτος 

print("Συνολικές τιμές:", grouped_single.shape[0])

print("Οι έγκυρες τιμές είναι", len(grouped_single['PM10']) - grouped_single['PM2.5']['count'].le(18).sum())

if len(grouped_single['PM10']) - grouped_single['PM2.5']['count'].le(18).sum() > (0.9*grouped_single.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Συνολικές τιμές: 366
Οι έγκυρες τιμές είναι 333
Ικανοποιείται ο έλεγχος πληρότητας


In [108]:
# Οριακή τιμή έτους = 25 μg/m3 (με βάση τις ημερήσιες τιμές και όχι τις ωριαίες)

print("Η οριακή τιμή του έτους είναι", round(grouped_single['PM2.5']['mean'].mean(),1), "μg/m3")

if grouped_single['PM2.5']['mean'].mean() > 40:
    print("Παρατηρείται παραβίαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")

Η οριακή τιμή του έτους είναι 14.6 μg/m3
Δεν παρατηρείται υπέρβαση του ορίου


# O3

In [175]:
# Ημερήσιος μέσος όρος 8ώρου/ για να είναι τα δεδομένα πλήρη απαιτούνται 18 8ωροι μέσοι όροι για κάθε μέρα (75%)

print("Οι συνολικές τιμές είναι", grouped_single.shape[0] )
print("Οι έγκυρες τιμές είναι", len(grouped_single['O3roll']) - grouped_single['O3roll']['count'].le(18).sum())

print(grouped_single['O3roll']['count'].le(18).sum())

#pd.set_option('display.max_rows', 366)
#print(grouped_single['O3roll']['count'])

# different results from excel 

Οι συνολικές τιμές είναι 366
Οι έγκυρες τιμές είναι 357
9


In [168]:
# Τιμή ορίου μέγιστου ημερήσιου 8ωρου: 120μg/m3 έως 25 φορές τον χρόνο σε διάστημα 3 χρόνων

print("Η οριακή τιμή του έτους είναι", round(grouped_single['O3roll']['max'].max(), 1), "μg/m3")

if grouped_single['O3roll']['max'].max() > 120 and grouped_single['O3roll']['count'].gt(18): #25 
    print("Παρατηρείται παραβίαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές παραβιάσεις:", grouped_single['O3roll']['max'].gt(120).sum() )

Η οριακή τιμή του έτους είναι 108.9 μg/m3
Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές παραβιάσεις: 0


In [169]:
# Όριο συναγερμού: 240 μg/m3 για 3 συνεχόμενες ώρες

times = 0
size = len(data['O3'])
  
for i in range(size - 2):
  
    if data['O3'][i] > 240 and data['O3'][i + 1] > 240  and data['O3'][i + 2] > 240:
        times += 1
        
print("Yπερβάσεις του ορίου συναργερμού που παρατηρήθηκαν:", times)

Yπερβάσεις του ορίου συναργερμού που παρατηρήθηκαν: 0


# CO

the data for this pollutant are not for the station of Pireus but for the station of Patision, Greece

In [112]:
# Ημερήσιος μέσος όρος 8ώρου/ για να είναι τα δεδομένα πλήρη απαιτούνται 18 8ωροι μέσοι όροι για κάθε μέρα (75%)

print("Οι συνολικές τιμές είναι", grouped_single.shape[0] )
print("Οι έγκυρες τιμές είναι", len(grouped_single['COroll']) - grouped_single['COroll']['count'].le(18).sum())

print(grouped_single['COroll']['count'].le(18).sum())

pd.set_option('display.max_rows', 50)
#print(grouped_single['COroll']['count'])

# different results from excel 

Οι συνολικές τιμές είναι 366
Οι έγκυρες τιμές είναι 332
34


In [113]:
# Τιμή ορίου μέγιστου ημερήσιου 8ωρου: 10mg/m3 

print("Η οριακή τιμή του έτους είναι", round(grouped_single['COroll']['max'].max(), 2), "mg/m3")

if grouped_single['COroll']['max'].max() > 10 and grouped_single['COroll']['count'].gt(18):
    print("Παρατηρείται παραβίαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές παραβιάσεις:", grouped_single['COroll']['max'].gt(10).sum() )

Η οριακή τιμή του έτους είναι 5.27 mg/m3
Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές παραβιάσεις: 0
